In [24]:
import numpy as np
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
import pprint
from IPython.display import display, IFrame
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id, client_secret= config.client_secret))

In [25]:
url = 'df_final.csv'
df = pd.read_csv(url)

In [28]:
results = {}

def start(start=True):
    global results  
    if start == True:
        song = input('Please input your Song: ')
    else:
        song = input('We could not find the correct song please try again: ')
    results = sp.search(q=song, limit=5)  # This modifies the global `results` variable
    print('We have found 5 songs. Please enter a number from 1 to 5, which song is yours (6 if the Song is not in the List):')
    print(' ')
    for index in range(3):
        print(f'Song {index + 1}:')
        track_id = results["tracks"]["items"][index]['id']
        iframe = IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
                        width="320",
                        height="80",
                        frameborder="0",
                        allowtransparency="true",
                        allow="encrypted-media")
        display(iframe)
    get_song()  



def get_song():
    song_index = input('Please input your Song number:')
    try:
        song_index = int(song_index) # Convert input to an integer
        if song_index == 4:
            start(start=False)
            return 
        else:
            # Make sure the song_index is within the valid range before accessing the list
            if 1 <= song_index <= 3:
                track_id = results["tracks"]["items"][song_index - 1]['id']
                run_model(track_id)
            else:
                print("Invalid input. Please enter a number between 1 and 3, or 4 if the song is not in the list.")
                get_song()
    except ValueError:
        # Handle the case where the input is not an integer
        print("Please enter a valid number.")
        get_song()


def run_model(track_id):
    global song_df
    input_track_audio = sp.audio_features(track_id)
    song_df = pd.DataFrame(input_track_audio)
    song_df = song_df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'loudness', 'mode', 'duration_ms'], axis=1)

    scaler = load("Model/scaler.pickle")
    y_scaled = scaler.transform(song_df)
    y_scaled_df = pd.DataFrame(y_scaled, columns=song_df.columns)

    kmeans = load('kmeans8.pickle')
    pred = kmeans.predict(y_scaled_df)[0]
    recomend_song(pred)

def recomend_song(pred):
    filtered_df = df[df['cluster'] == pred]
    random_song = filtered_df.sample(n=1)
    recommended_id = random_song['id'].values[0]
    print('We have a really good song recommendation for you:')
    print(' ')
    iframe = IFrame(src=f"https://open.spotify.com/embed/track/{recommended_id}",
                        width="320",
                        height="80",
                        frameborder="0",
                        allowtransparency="true",
                        allow="encrypted-media")
    display(iframe)

def load(filename = "kmeans8.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

In [29]:
start()

Please input your Song:  hello


We have found 5 songs. Please enter a number from 1 to 5, which song is yours (6 if the Song is not in the List):
 
Song 1:


Song 2:


Song 3:


Please input your Song number: 1


We have a really good song recommendation for you:
 
